In [ ]:
import pandas as pd
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize


In [ ]:
import requests

ACCOUNT_ADDRESS = 'rEb8TK3gBgk5auZkwc6sHnwrGVJH8DuaLh'
url = 'https://api.xrpscan.com/api/v1/account//{ACCOUNT_ADDRESS}'.format(ACCOUNT_ADDRESS=ACCOUNT_ADDRESS)

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_json(r.text)
dfs.fillna('', inplace=True)
dfs

In [ ]:
from IPython.display import JSON
JSON(r.text, expanded=True)

# Get Payment flows

In [ ]:
ACCOUNT_ADDRESS = 'rEb8TK3gBgk5auZkwc6sHnwrGVJH8DuaLh'
url = 'https://api.xrpscan.com/api/v1/account/{ACCOUNT_ADDRESS}/payment_flows'.format(ACCOUNT_ADDRESS=ACCOUNT_ADDRESS)

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
JSON(r.text, expanded=True)

# Get ALL Name from TAG

In [ ]:
url = 'https://api.xrpscan.com/api/v1/names/well-known'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
JSON(r.text, expanded=True)

In [ ]:
import requests
import json
from pandas.io.json import json_normalize

url = 'https://api.xrpscan.com/api/v1/names/well-known'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

data = json.loads(r.text)
df = json_normalize(data)
df.fillna('', inplace=True)
df.loc[df.name.str.contains('Binance')]

In [ ]:
df.to_excel('data/List of well-known Names used to identify accounts.xlsx')

# Get Account Balances

In [ ]:
import pandas as pd
import requests
import json
from urllib.request import urlopen
from IPython.display import JSON
from pandas.io.json import json_normalize

from datetime import datetime
from datetime import timedelta

now = datetime.now() - timedelta(days=1)
date_time = now.strftime("%Y-%m-%d")

ACCOUNT_ADDRESS = 'rEeEWeP88cpKUddKk37B2EZeiHBGiBXY3'
url = 'https://data.ripple.com/v2/accounts/{ACCOUNT_ADDRESS}/balances?currency=XRP&date={date_time}T00:00:00Z&limit=3'.format(ACCOUNT_ADDRESS=ACCOUNT_ADDRESS, date_time=date_time)

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

data = json.loads(r.text)
JSON(data, expanded=True)

# Get daily reports

In [ ]:
import pandas as pd
import requests
import json
from urllib.request import urlopen
from IPython.display import JSON
from pandas.io.json import json_normalize

from datetime import datetime
from datetime import timedelta

now = datetime.now() - timedelta(days=1)
date_time = now.strftime("%Y-%m-%d")

url = 'https://data.ripple.com/v2/reports/{date_time}T00:00:00Z?accounts=true&payments=true'.format(date_time=date_time)

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

data = json.loads(r.text)
JSON(data, expanded=True)

# Get balance

In [ ]:
import pandas as pd
import requests
import json
from urllib.request import urlopen
from IPython.display import JSON
from pandas.io.json import json_normalize

from datetime import datetime
from datetime import timedelta
import time

df_name = pd.read_excel('data/List of well-known Names used to identify accounts.xlsx')
df_name.fillna('', inplace=True)

now = datetime.now()
date_time = now.strftime("%Y-%m-%d")

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

df_xrp = df_name.copy(deep=True)

print('Begin')

df_name = df_name.sort_values(by='XRP', ascending=False)
df_name = df_name[:100] # get top 30 balance

for _, row in df_name.iterrows():
    print('{}-{}-{}'.format(row['account'], row['name'], row['desc']))

    ACCOUNT_ADDRESS = row['account']
    
    url = 'https://data.ripple.com/v2/accounts/{ACCOUNT_ADDRESS}/balances?currency=XRP&date={date_time}T00:00:00Z&limit=3'.format(ACCOUNT_ADDRESS=ACCOUNT_ADDRESS, date_time=date_time)

    r = requests.get(url, headers=header)

    data = json.loads(r.text)
    if data['result'] != 'error':
        df_xrp.loc[df_xrp['account'] == ACCOUNT_ADDRESS, date_time] = float(data['balances'][0]['value'])
        
    #print('-----------------------')
    time.sleep(0.001)   # Delays for 1 seconds. You can also use a float value.

df_xrp['verified'] = df_xrp['verified'].astype(bool)

df_xrp.to_excel('data/{}-List of well-known Names used to identify accounts.xlsx'.format(date_time), index=False)

print('Done.')

In [ ]:
df_xrp.to_excel('data/List of well-known Names used to identify accounts.xlsx')

# Get Account Transaction History

In [ ]:
url = 'https://data.ripple.com/v2/accounts/rsdvR9WZzKszBogBJrpLPE64WWyEW4ffzS/transactions?type=Payment&result=tesSUCCESS&limit=10'

In [ ]:
import pandas as pd
import requests
import json
from urllib.request import urlopen
from IPython.display import JSON
from pandas.io.json import json_normalize

ACCOUNT_ADDRESS = 'rw7m3CtVHwGSdhFjV4MyJozmZJv3DYQnsA'
url = 'https://data.ripple.com/v2/accounts/{ACCOUNT_ADDRESS}/transactions?type=Payment&result=tesSUCCESS&limit=400'.format(ACCOUNT_ADDRESS=ACCOUNT_ADDRESS)

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

data = json.loads(r.text)
JSON(data, expanded=True)

In [ ]:
JSON(data['transactions'][17], expanded=True)

# Get Transactions

## Config

In [ ]:
import pandas as pd
import requests
import json
from urllib.request import urlopen
from IPython.display import JSON
from pandas.io.json import json_normalize

from datetime import datetime
from datetime import timedelta
import time

## Load Account Name

In [ ]:
df_name = pd.read_excel('data/List of well-known Names used to identify accounts.xlsx')
df_name.fillna('', inplace=True)
df_name = df_name.sort_values(by='XRP', ascending=False)
df_name.head()

In [ ]:
frames = []
df_name.fillna('', inplace=True)
for _, row in df_name.iterrows():
    print('{}-{}-{}'.format(row['account'], row['name'], row['desc']))
    ACCOUNT_ADDRESS = row['account']
    data = load_transactions(ACCOUNT_ADDRESS)
    
    if data.__contains__('result'):
        if data['result'] == 'success' and data['count'] > 0:
            df_tran = convert_to_dataframe(ACCOUNT_ADDRESS, data)
            frames.append(df_tran)

df_trans = pd.concat(frames)
df_trans.tail()

In [ ]:
ACCOUNT_ADDRESS = 'rNh4YSHxzuPb3QhcV7EWime6Jc5C8YcRYH'
data = load_transactions(ACCOUNT_ADDRESS)
df_tran = convert_to_dataframe(ACCOUNT_ADDRESS, data)
JSON(data, expanded=True)
#df_tran

In [ ]:
JSON(data['transactions'][0], expanded=True)

## Export To Excel

In [ ]:
df_trans.to_excel('data/transactions.xlsx', index=False)

## Load transactions

In [ ]:
def load_transactions(ACCOUNT_ADDRESS):
    start = datetime.now() - timedelta(days=10)
    date_start = start.strftime("%Y-%m-%dT%H:%M:%SZ")

    end = datetime.now()
    date_end = end.strftime("%Y-%m-%dT%H:%M:%SZ")

    # ACCOUNT_ADDRESS = 'rw7m3CtVHwGSdhFjV4MyJozmZJv3DYQnsA'
    url = 'https://data.ripple.com/v2/accounts/{ACCOUNT_ADDRESS}/transactions?start={date_start}&end={date_end}&type=Payment&result=tesSUCCESS&limit=400&descending=True'.format(ACCOUNT_ADDRESS=ACCOUNT_ADDRESS, date_start=date_start, date_end=date_end)

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    data = json.loads(r.text)
    # JSON(data, expanded=True)
    return data

## Convert to DataFrame

In [ ]:
def convert_to_dataframe(account, data):
    if data['count'] == 0:
        return
    
    trans_list=[]
    for d in data['transactions']:
        account_desc = ''
        d_date = d['date']
        d_hash = d['hash']
        d_tx_from = d['tx']['Account']
        d_tx_from_desc = ''
        d_tx_type = d['tx']['TransactionType']
        d_tx_flow = ''
        d_tx_to = d['tx']['Destination']
        d_tx_to_desc = ''
        d_tx_dt = d['tx']['DestinationTag'] if d['tx'].__contains__('DestinationTag') else ''
        
        d_tx_amount = 0
        d_tx_currency = ''
        d_tx_issuer = ''
        if str(type(d['tx']['Amount'])) != "<class 'dict'>":
            d_tx_amount = float(d['tx']['Amount'])  / 1000000
        else:
            d_tx_amount = float(d['tx']['Amount']['value']) / 1000000
            d_tx_currency = d['tx']['Amount']['currency'][0:8]
            d_tx_issuer = d['tx']['Amount']['issuer']
        
        d_meta_delivered_amount = 0
        if str(type(d['meta']['delivered_amount'])) != "<class 'dict'>":
            d_meta_delivered_amount =  float(d['meta']['delivered_amount'])  / 1000000
        else:
            d_meta_delivered_amount =  float(d['meta']['delivered_amount']['value'])  / 1000000
            
        d_tx_fee = float(d['tx']['Fee']) / 1000000
        d_meta_result = d['meta']['TransactionResult']

        trans_list.append([account, account_desc, d_date, d_hash, d_tx_from, d_tx_from_desc, d_tx_type, d_tx_flow, d_tx_to, d_tx_to_desc, d_tx_dt, d_tx_amount, d_tx_currency, d_tx_issuer, d_meta_delivered_amount, d_tx_fee, d_meta_result])

    df_trans = pd.DataFrame(trans_list, columns=['account','Account Desc', 'Date','Tx hash','From','From Desc','Type','Flow','To','To Desc','DT','Amount', 'Currency', 'Issuer', 'Delivered Amount', 'Fee', 'Result'])
    
    df_trans['Account Desc'] = df_trans['account'].apply(lambda x: find_desc(x))
    df_trans['From Desc'] = df_trans['From'].apply(lambda x: find_desc(x))
    df_trans['To Desc'] = df_trans['To'].apply(lambda x: find_desc(x))
    df_trans['Flow'] = df_trans[['account', 'To']].apply(lambda x: 'IN' if x['account'] == x['To'] else 'OUT', axis=1)
    
    return df_trans


## Find Account Description

In [ ]:
def find_desc(account):
    try:
        _df = df_name.loc[df_name.account == account, ['name','desc']].astype(str)
        return '{}{}'.format(_df.name.values[0], '({})'.format(_df.desc.values[0]) if len(_df.desc.values[0]) > 0 else '')
    except:
        return 'Unknow'
    